In [38]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch.optim as optim
from torch.autograd import Variable
from typing import Dict, Any, Tuple, Union, NamedTuple
import hydra

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lr = 1e-3
lbd_lr = 1e-2
max_steps = 20000
print_freq = 100

#8 layers, 256 neurons test.

torch.manual_seed(995) 

In [39]:

lc = 1e-9
cc = 1e-5
# Physical constants
F = 96485 # Faraday constant [C/mol]
R = 8.3145 # Gas constant [J/(mol·K)]
T = 293 # Temperature [K]
k_B = 1.3806e-23 # Boltzmann constant [J/K]
eps0 = 8.85e-12 # Vacuum permittivity [F/m]
E_min = -1.0  # External applied potential [V] 
E_max = 1.8
# Diffusion coefficients [m²/s]
D_cv = 1.0e-21
D_av = 1.0e-21
D_h = 3.2823e-4

# Mobility coefficients [m²/(V·s)]
U_cv = -1.0562e-19
U_av = 7.9212e-20
U_h = 0.013 # mo_h from COMSOL

# Species charges
z_cv = -2.6667 # -8/3
z_av = 2
z_h = 1

# Permittivities [F/m]
epsilonf = 1.239e-10 # 14*eps0
eps_film = 1.239e-10 # Same as epsilonf
eps_Ddl = 1.77e-11 # 2*eps0
eps_dl = 6.947e-10 # 78.5*eps0
eps_sol = 6.947e-10 # Same as eps_dl

# Semiconductor properties
c_h0 = 4.1683e-4 # Intrinsic hole concentration [mol/m³]
c_e0 = 9.5329e-28 # Intrinsic electron concentration [mol/m³]
tau = 4.9817e-13 # Recombination time constant [s·mol/m³]
Nc = 166.06 # Conduction band density [mol/m³]
Nv = 1.6606e5 # Valence band density [mol/m³]
mu_e0 = 2.4033e-19 # Standard electron chemical potential [J]
Ec0 = 5.127e-19 # Conduction band edge [J]
Ev0 = 1.6022e-19 # Valence band edge [J]

# Solution properties
c_H = 0.01 # Proton concentration [mol/m³]
pH = 5

# Molar volume
Omega = 1.4e-5 # [m³/mol]
# Standard rate constants
k1_0 = 4.5e-8 # [m/s]
k2_0 = 3.6e-6 # [mol/(m²·s)]
k3_0 = 4.5e-9 # [mol/(m²·s)]
k4_0 = 2.25e-7 # [m/s]
k5_0 = 7.65e-9 # [mol/(m²·s)]
ktp_0 = 4.5e-8 # [-]
ko2_0 = 0.005 # [m/s]

# Charge transfer coefficients
alpha_cv = 0.3
alpha_av = 0.8
beta_cv = 0.1
beta_av = 0.8
alpha_tp = 0.2
a_par = 0.45 # For oxygen evolution
delta3 = 1.0

# Derived parameters [1/V]
a_cv = 23.764 # alpha_cv * 2 * F/(R*T)
a_av = 84.493 # alpha_av * 8/3 * F/(R*T)
b_cv = 7.9212 # beta_cv * 2 * F/(R*T)

# Geometric parameters [m]
d_Ddl = 2.0e-10 # Defect layer thickness
d_dl = 5.0e-10 # Double layer thickness
L_cell = 1.0e-6 # Cell length

# Equilibrium potentials
phi_O2_eq = 1.35 # [V]
phic = (R*T)/F
tc = (lc ** 2) / D_cv
L_initial = 1e-10

In [40]:
#Define networks

class Swish(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self,x):
        return torch.sigmoid(x)*x
    

class FFN(nn.Module):
    """
    Fully Connected Feed Forward Neural Network.
    Args:
        input_dim: Number of input features
        output_dim: Number of output features  
        hidden_layers: Number of hidden layers
        layer_size: Size of each hidden layer
        activation: Activation function name ('swish', 'swoosh', 'swash', 'squash_swish', 'relu', 'tanh')
        initialize_weights: Whether to apply Xavier initialization
    """
    def __init__(
        self,
        input_dim: int = 3,
        output_dim: int = 1,
        hidden_layers: int = 8,
        layer_size: int = 256,
        activation: str = "swish",
        initialize_weights: bool = False
    ):
        super(FFN, self).__init__()
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_layers = hidden_layers
        self.layer_size = layer_size
        self.activation = Swish()
        
        # Input layer
        self.input_layer = nn.Linear(input_dim, self.layer_size)
        
        # Hidden layers
        self.hidden_layers = nn.ModuleList([
            nn.Linear(self.layer_size, self.layer_size)
            for _ in range(self.num_layers)  
        ])
        
        # Output layer
        self.output_layer = nn.Linear(self.layer_size, output_dim)
        
        # Initialize weights
        if initialize_weights:
            self.initialize_weights()
    
    def initialize_weights(self):
        """Apply Xavier initialization to all linear layers"""
        # Initialize input layer
        nn.init.xavier_normal_(self.input_layer.weight)
        nn.init.zeros_(self.input_layer.bias)
        
        # Initialize hidden layers
        for layer in self.hidden_layers:
            nn.init.xavier_normal_(layer.weight)
            nn.init.zeros_(layer.bias)
        
        # Initialize output layer
        nn.init.xavier_normal_(self.output_layer.weight)
        nn.init.zeros_(self.output_layer.bias)
    
    def forward(self, x):
        x = self.activation(self.input_layer(x))
        
        for layer in self.hidden_layers: 
            x = self.activation(layer(x))
        
        return self.output_layer(x)


cv_net = FFN()
av_net = FFN()
u_net = FFN()
L_net = FFN(2,1)
cv_net.to(device)
av_net.to(device)
u_net.to(device)
L_net.to(device)

total_model_parameters = list(cv_net.parameters()) + list(av_net.parameters()) + list(u_net.parameters()) + list(L_net.parameters()) 

In [41]:
#gradient and sampling utils

class GradientResults(NamedTuple):
    """
    Container for gradient computation results.

    Organizes all computed derivatives in a structured way for easy access.
    """
    # Network predictions
    phi: torch.Tensor  # Potential φ
    c_cv: torch.Tensor  # Cation vacancy concentration
    c_av: torch.Tensor  # Anion vacancy concentration

    # Time derivatives
    c_cv_t: torch.Tensor  # ∂c_cv/∂t
    c_av_t: torch.Tensor  # ∂c_av/∂t

    # First spatial derivatives
    phi_x: torch.Tensor  # ∂φ/∂x
    c_cv_x: torch.Tensor  # ∂c_cv/∂x
    c_av_x: torch.Tensor  # ∂c_av/∂x

    # Second spatial derivatives
    phi_xx: torch.Tensor  # ∂²φ/∂x²
    c_cv_xx: torch.Tensor  # ∂²c_cv/∂x²
    c_av_xx: torch.Tensor  # ∂²c_av/∂x²


def _grad(x,dx):
    """Take the derrivative of x w.r.t dx"""

    return torch.autograd.grad(x,dx,torch.ones_like(dx),create_graph=True,retain_graph=True)[0]

def compute_gradients(x, t, E):
    inputs_3d = torch.cat([x, t, E], dim=1)

    # Get network predictions
    phi = u_net(inputs_3d)
    c_cv_raw = cv_net(inputs_3d)
    c_av_raw = av_net(inputs_3d)


    # Networks predict concentrations directly
    c_cv = c_cv_raw
    c_av = c_av_raw


    # Direct derivatives
    c_cv_t = _grad(c_cv, t)
    c_av_t = _grad(c_av, t)

    # Compute first spatial derivatives
    phi_x = _grad(phi, x)

    c_cv_x = _grad(c_cv, x)
    c_av_x = _grad(c_av, x)

    # Compute second spatial derivatives
    phi_xx = _grad(phi_x, x)

    c_cv_xx = _grad(c_cv_x, x)
    c_av_xx = _grad(c_av_x, x)

    return GradientResults(
        phi=phi, c_cv=c_cv, c_av=c_av,
        c_cv_t=c_cv_t, c_av_t=c_av_t,
        phi_x=phi_x, c_cv_x=c_cv_x, c_av_x=c_av_x, 
        phi_xx=phi_xx, c_cv_xx=c_cv_xx, c_av_xx=c_av_xx
    )

def sample_interior_points(
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Sample interior collocation points for PDE residuals.

        Args:
            networks: NetworkManager instance

        Returns:
            Tuple of (x, t, E) tensors with requires_grad=True for x and t
        """
        batch_size = 2048

        # Sample time and applied potential
        t = torch.rand(batch_size, 1, device=device, requires_grad=True)
        single_E = torch.tensor(0.8,device=device)
        E = single_E.expand(batch_size, 1)

        # Get film thickness prediction
        L_pred = L_net(torch.cat([t, E], dim=1))

        # Sample spatial coordinates within [0, L(t,E)]
        x = torch.rand(batch_size, 1, device=device, requires_grad=True) * L_pred

        return x, t, E

def sample_boundary_points(
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Sample boundary collocation points for boundary conditions.

    Args:
        networks: NetworkManager instance

    Returns:
        Tuple of (x, t, E) tensors for boundary points
    """
    batch_size = 2 * 1024

    # Sample time and applied potential
    t = torch.rand(batch_size, 1, device=device, requires_grad=True)
    single_E = torch.tensor(0.8,device=device)
    E = single_E.expand(batch_size, 1)

    # Predict L for f/s boundary
    L_inputs = torch.cat([t, E], dim=1)
    L_pred = L_net(L_inputs)

    half_batch = batch_size // 2

    # Metal/film interface points (x = 0)
    x_mf = torch.zeros(half_batch, 1, device=device, requires_grad=True)
    t_mf = t[:half_batch]
    E_mf = E[:half_batch]

    # Film/solution interface points (x = L)
    x_fs = L_pred[half_batch:]
    t_fs = t[half_batch:]
    E_fs = E[half_batch:]

    # Combine boundary points
    x_boundary = torch.cat([x_mf, x_fs], dim=0)
    t_boundary = torch.cat([t_mf, t_fs], dim=0)
    E_boundary = torch.cat([E_mf, E_fs], dim=0)

    return x_boundary, t_boundary, E_boundary

def sample_initial_points(
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Sample initial condition points at t = 0.

    Args:
        networks: NetworkManager instance

    Returns:
        Tuple of (x, t, E) tensors for initial condition points
    """
    batch_size = 1024

    # Initial time (t = 0)
    t = torch.zeros(batch_size, 1, device=device, requires_grad=True)
    single_E = torch.tensor(0.8,device=device)
    E = single_E.expand(batch_size, 1)

    # Get initial film thickness
    L_initial_pred = L_net(torch.cat([t, E], dim=1))

    # Sample spatial coordinates
    x = (
        torch.rand(batch_size, 1, device=device, requires_grad=True)
        * L_initial_pred
    )

    return x, t, E

def sample_film_physics_points() -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Sample points for film growth physics constraint.

    Returns:
        Tuple of (t, E) tensors for film physics constraint
    """
    batch_size = 2048

    # Sample time and applied potential
    t = torch.rand(batch_size, 1, device=device, requires_grad=True)
    single_E = torch.tensor(0.8,device=device)
    E = single_E.expand(batch_size, 1)

    return t, E


In [42]:
#Compute and generate all the relevant losses

def compute_rate_constants(t: torch.Tensor, E: torch.Tensor, single: bool = False):
        """
        Compute electrochemical rate constants using Butler-Volmer kinetics.

        **Butler-Volmer Rate Expressions:**

        .. math::
            \\hat{k}_{R1} = k_1^0 \\exp\\left(\\alpha_1 \\frac{3F\\hat{\\phi}_c}{RT}\\hat{\\phi}_{mf} \\right)

        .. math::
            \\hat{k}_{R2} = k_2^0 \\exp\\left(\\alpha_2 \\frac{2F\\hat{\\phi}_c}{RT}\\hat{\\phi}_{mf} \\right)

        .. math::
            \\hat{k}_{R3} = k_3^0 \\exp\\left(\\alpha_3 \\frac{(3-\\delta)F\\hat{\\phi}_c}{RT}\\hat{\\phi}_{fs} \\right)

        .. math::
            \\hat{k}_{R4} = k_4^0

        .. math::
            \\hat{k}_{R5} = k_5^0 (c_{H^+})^n

        .. math::
            \\hat{k}_{TP} = k_{tp}^0 \\hat{c}_h c_c \\exp\\left(\\alpha_{tp}\\frac{F\\hat{\\phi}_c}{RT}\\hat{\\phi}_{fs}\\right)

        .. math::
            \\hat{k}_{O2} = k_{o2}^0 \\exp\\left(\\alpha_{o2}\\frac{2F\\hat{\\phi}_c}{RT} \\left(\\hat{\\phi}_{ext} - \\hat{\\phi}_{o2,eq}\\right) \\right)

        where:
        - :math:`\\hat{\\phi}_{mf}` is the dimensionless potential at metal/film interface
        - :math:`\\hat{\\phi}_{fs}` is the dimensionless potential at film/solution interface
        - :math:`\\alpha_i, \\beta_i` are charge transfer coefficients

        Args:
            t: Time tensor (dimensionless)
            E: Applied potential tensor
            networks: NetworkManager instance
            single: Whether computing for single point or batch

        Returns:
            Tuple of rate constants (k1, k2, k3, k4, k5, ktp, ko2)
        """
        if single:
            batch_size = 1
            x_mf = torch.zeros(1, 1, device=device)
        else:
            batch_size = t.shape[0]
            x_mf = torch.zeros(batch_size, 1, device=device)

        # Get potentials at interfaces
        inputs_mf = torch.cat([x_mf, t, E], dim=1)
        u_mf = u_net(inputs_mf)  # φ̂_mf

        # Get film thickness
        L_inputs = torch.cat([t, E], dim=1)
        L_pred = L_net(L_inputs)


        x_fs = L_pred

        inputs_fs = torch.cat([x_fs, t, E], dim=1)
        u_fs = u_net(inputs_fs)  # φ̂_fs

        # Compute rate constants using equations above
        F_RT = F * phic / (R * T)

        # k₁: Cation vacancy generation at m/f interface
        k1 = k1_0 * torch.exp(alpha_cv * 3 * F_RT * u_mf)

        # k₂: Anion vacancy generation at m/f interface
        k2 = k2_0 * torch.exp(alpha_av * 2 * F_RT * u_mf)

        # k₃: Cation vacancy consumption at f/s interface
        k3 = k3_0 * torch.exp(beta_cv * (3 - delta3) * F_RT * u_fs)

        # k₄: Chemical reaction (potential independent)
        k4 = k4_0

        # k₅: Chemical dissolution
        k5 = k5_0 * c_H

        return k1, k2, k3, k4, k5

def compute_pde_residuals(x: torch.Tensor, t: torch.Tensor, E: torch.Tensor):
        """
        Compute PDE residuals for all governing equations.

        **Cation Vacancy Conservation (Dimensionless Nernst-Planck):**

        .. math::
            \\frac{\\partial \\hat{c}_{cv}}{\\partial \\hat{t}} =
            \\frac{D_{cv}\\hat{t}_c}{\\hat{L}_c^2}\\frac{\\partial^2 \\hat{c}_{cv}}{\\partial \\hat{x}^2} +
            \\frac{U_{cv}\\hat{t}_c\\hat{\\phi}_c}{\\hat{L}_c^2}\\frac{\\partial \\hat{\\phi}}{\\partial \\hat{x}}\\frac{\\partial \\hat{c}_{cv}}{\\partial \\hat{x}} +
            \\frac{U_{cv}\\hat{t}_c\\hat{\\phi}_c}{\\hat{L}_c^2}\\hat{c}_{cv}\\frac{\\partial^2 \\hat{\\phi}}{\\partial \\hat{x}^2}

        **Anion Vacancy Conservation (Dimensionless Nernst-Planck):**

        .. math::
            \\frac{\\partial \\hat{c}_{av}}{\\partial \\hat{t}} =
            \\frac{D_{av}\\hat{t}_c}{\\hat{L}_c^2}\\frac{\\partial^2 \\hat{c}_{av}}{\\partial \\hat{x}^2} +
            \\frac{U_{av}\\hat{t}_c\\hat{\\phi}_c}{\\hat{L}_c^2}\\frac{\\partial \\hat{\\phi}}{\\partial \\hat{x}}\\frac{\\partial \\hat{c}_{av}}{\\partial \\hat{x}} +
            \\frac{U_{av}\\hat{t}_c\\hat{\\phi}_c}{\\hat{L}_c^2}\\hat{c}_{av}\\frac{\\partial^2 \\hat{\\phi}}{\\partial \\hat{x}^2}

        **Hole Conservation (Quasi-Steady State):**

        .. math::
            0 = \\frac{D_h\\hat{c}_{h,c}}{\\hat{L}_c^2}\\frac{\\partial^2 \\hat{c}_h}{\\partial \\hat{x}^2} +
            \\frac{FD_h\\hat{\\phi}_c\\hat{c}_{h,c}}{RT\\hat{L}_c^2}\\frac{\\partial \\hat{\\phi}}{\\partial \\hat{x}}\\frac{\\partial \\hat{c}_h}{\\partial \\hat{x}} +
            \\frac{FD_h\\hat{\\phi}_c\\hat{c}_{h,c}}{RT\\hat{L}_c^2}\\hat{c}_h\\frac{\\partial^2 \\hat{\\phi}}{\\partial \\hat{x}^2}

        **Poisson's Equation (Dimensionless):**

        .. math::
            \\frac{\\partial^2 \\hat{\\phi}}{\\partial \\hat{x}^2} =
            -\\frac{F\\hat{L}_c^2\\hat{c}_c}{\\hat{\\phi}_c\\varepsilon_f}\\left(z_{av}\\hat{c}_{av} + z_{cv}\\hat{c}_{cv}\\right)

        Args:
            x: Spatial coordinates (dimensionless)
            t: Time coordinates (dimensionless)
            E: Applied potential
            networks: NetworkManager instance

        Returns:
            Tuple of residuals: (cv_residual, av_residual, h_residual, poisson_residual)
        """
        # Get all gradients using the gradient computer
        grads = compute_gradients(x, t, E)

        # Cation vacancy residual - implements equation above
        cv_residual = (grads.c_cv_t -
                       (D_cv * tc / lc ** 2) * grads.c_cv_xx -
                       (U_cv * tc * phic / lc ** 2) *
                       grads.phi_x * grads.c_cv_x -
                       (U_cv * tc * phic / lc ** 2) *
                       grads.c_cv * grads.phi_xx)

        # Anion vacancy residual - implements equation above
        av_residual = (grads.c_av_t -
                       (D_av * tc / lc ** 2) * grads.c_av_xx -
                       (U_av * tc * phic / lc ** 2) *
                       grads.phi_x * grads.c_av_x -
                       (U_av * tc * phic / lc ** 2) *
                       grads.c_av * grads.phi_xx)

        # Poisson residual - implements equation above
        poisson_residual = (grads.phi_xx +
                            (F * lc ** 2 * cc /
                             (phic * epsilonf)) *
                            (z_av * grads.c_av + z_cv * grads.c_cv))

        return cv_residual, av_residual, poisson_residual


def compute_interior_loss(x: torch.Tensor, t: torch.Tensor, E: torch.Tensor,
                          return_residuals: bool = False) -> Union[Tuple[torch.Tensor, Dict[str, torch.Tensor]],
Tuple[torch.Tensor, Dict[str, torch.Tensor], Dict[str, torch.Tensor]]]:
    """
    Compute interior PDE residual losses.

    See compute_pde_residuals for mathematics of residual calculations

    Args:
        x: Spatial coordinates
        t: Time coordinates
        E: Applied potential
        networks: NetworkManager instance
        physics: ElectrochemicalPhysics instance
        return_residuals: If True, also return raw residuals for NTK computation

    Returns:
        If return_residuals=False: Tuple of (total_interior_loss, individual_losses_dict)
        If return_residuals=True: Tuple of (total_interior_loss, individual_losses_dict, residuals_dict)
    """
    # Compute PDE residuals using physics module
    cv_residual, av_residual, poisson_residual = compute_pde_residuals(x, t, E)

    # Calculate individual losses
    cv_pde_loss = torch.mean(cv_residual ** 2)
    av_pde_loss = torch.mean(av_residual ** 2)
    poisson_pde_loss = torch.mean(poisson_residual ** 2)

    # Total interior loss
    total_interior_loss = cv_pde_loss + av_pde_loss + poisson_pde_loss

    individual_losses = {
        'cv_pde': cv_pde_loss,
        'av_pde': av_pde_loss,
        'poisson_pde': poisson_pde_loss
    }

    if return_residuals:
        residuals = {
            'cv_pde': cv_residual,
            'av_pde': av_residual,
            'poisson_pde': poisson_residual
        }
        return total_interior_loss, individual_losses, residuals
    else:
        return total_interior_loss, individual_losses

def compute_boundary_loss(x: torch.Tensor, t: torch.Tensor, E: torch.Tensor,
                          return_residuals: bool = False) -> Union[Tuple[torch.Tensor, Dict[str, torch.Tensor]],
Tuple[torch.Tensor, Dict[str, torch.Tensor], torch.Tensor]]:
    """
    Compute boundary condition losses.
     **Boundary Conditions:**

    **Metal/Film Interface (x̂ = 0):**

    *Cation Vacancy Flux:*

    .. math::
        -D_{cv}\\frac{\\partial \\hat{c}_{cv}}{\\partial \\hat{x}} = \\hat{k}_1 - \\left(U_{cv}\\frac{\\partial \\hat{\\phi}}{\\partial \\hat{x}} - \\frac{d\\hat{L}}{d\\hat{t}}\\right)\\hat{c}_{cv}

    *Anion Vacancy Flux:*

    .. math::
        -D_{av}\\frac{\\partial \\hat{c}_{av}}{\\partial \\hat{x}} = \\frac{4}{3}\\hat{k}_2 + \\left(U_{av}\\frac{\\partial \\hat{\\phi}}{\\partial \\hat{x}} - \\frac{d\\hat{L}}{d\\hat{t}}\\right)\\hat{c}_{av}

    *Potential Boundary:*

    .. math::
        \\varepsilon_f \\frac{\\partial \\hat{\\phi}}{\\partial \\hat{x}} = \\frac{\\varepsilon_{Ddl}(\\hat{\\phi} - \\hat{E})}{\\hat{d}_{Ddl}}

    **Film/Solution Interface (x̂ = L̂):**

    *Cation Vacancy Flux:*

    .. math::
        -D_{cv}\\frac{\\partial \\hat{c}_{cv}}{\\partial \\hat{x}} = \\left(\\hat{k}_3 - U_{cv}\\frac{\\partial \\hat{\\phi}}{\\partial \\hat{x}}\\right)\\hat{c}_{cv}

    *Anion Vacancy Flux:*

    .. math::
        -D_{av}\\frac{\\partial \\hat{c}_{av}}{\\partial \\hat{x}} = \\left(\\hat{k}_4 - U_{av}\\frac{\\partial \\hat{\\phi}}{\\partial \\hat{x}}\\right)\\hat{c}_{av}

    *Hole Flux:*

    .. math::
        D_h\\frac{\\partial \\hat{c}_h}{\\partial \\hat{x}} = \\hat{q}\\hat{c}_h

    where :math:`\\hat{q} = -(\\hat{k}_{tp} + \\frac{FD_h}{RT}\\frac{\\partial \\hat{\\phi}}{\\partial \\hat{x}})` for :math:`\\hat{c}_h > 10^{-9}`

    *Potential Boundary:*

    .. math::
        \\varepsilon_f \\frac{\\partial \\hat{\\phi}}{\\partial \\hat{x}} = \\varepsilon_{Ddl}\\hat{\\phi}
    Args:
        x: Boundary spatial coordinates
        t: Time coordinates
        E: Applied potential
        networks: NetworkManager instance
        physics: ElectrochemicalPhysics instance
        return_residuals: If True, also return raw residuals for NTK computation

    Returns:
        If return_residuals=False: Tuple of (total_boundary_loss, individual_losses_dict)
        If return_residuals=True: Tuple of (total_boundary_loss, individual_losses_dict, combined_residuals)
    """
    batch_size = x.shape[0]
    half_batch = batch_size // 2

    # Split into metal/film and film/solution interfaces
    x_mf = x[:half_batch]
    x_fs = x[half_batch:]
    t_mf = t[:half_batch]
    t_fs = t[half_batch:]
    E_mf = E[:half_batch]
    E_fs = E[half_batch:]

    # Predict L and compute derivative for boundary fluxes
    L_input = torch.cat([t, E], dim=1)
    L_pred = L_net(L_input)
    L_pred_t = _grad(L_pred, t)
    L_pred_t_mf = L_pred_t[:half_batch]

    # Metal/film interface conditions
    inputs_mf = torch.cat([x_mf, t_mf, E_mf], dim=1)
    u_pred_mf = u_net(inputs_mf)
    u_pred_mf_x = _grad(u_pred_mf, x_mf)

    # CV at m/f interface
    cv_pred_mf = cv_net(inputs_mf)
    cv_pred_mf_x = _grad(cv_pred_mf, x_mf)
    cv_mf_residual = ((-D_cv * cc / lc) * cv_pred_mf_x -
                      k1_0 * torch.exp(
                alpha_cv * phic * (E_mf / phic - u_pred_mf)) -
                      (U_cv * phic / lc * u_pred_mf_x -
                       lc / tc * L_pred_t_mf) * cc * cv_pred_mf)
    cv_mf_loss = torch.mean(cv_mf_residual ** 2)

    # AV at m/f interface
    av_pred_mf = av_net(inputs_mf)
    av_pred_mf_x = _grad(av_pred_mf, x_mf)
    av_mf_residual = ((-D_av * cc / lc) * av_pred_mf_x -
                      (4 / 3) * k2_0 * torch.exp(
                alpha_av * phic * (E_mf / phic - u_pred_mf)) -
                      (U_av * phic / lc * u_pred_mf_x -
                       lc / tc * L_pred_t_mf) * av_pred_mf)
    av_mf_loss = torch.mean(av_mf_residual ** 2)

    # Potential at m/f interface
    u_mf_residual = ((eps_film * phic /lc * u_pred_mf_x) -
                     eps_Ddl * phic * (
                             u_pred_mf - E_mf / phic) / d_Ddl)
    u_mf_loss = torch.mean(u_mf_residual ** 2)

    # Film/solution interface conditions
    inputs_fs = torch.cat([x_fs, t_fs, E_fs], dim=1)
    u_pred_fs = u_net(inputs_fs)
    u_pred_fs_x = _grad(u_pred_fs, x_fs)

    # CV at f/s interface
    cv_pred_fs = cv_net(inputs_fs)
    cv_pred_fs_x = _grad(cv_pred_fs, x_fs)
    cv_fs_residual = ((-D_cv * cc / lc) * cv_pred_fs_x -
                      (k3_0 * torch.exp(beta_cv * phic * u_pred_fs) -
                       U_cv * phic / lc * u_pred_fs_x) * cv_pred_fs * cc)
    cv_fs_loss = torch.mean(cv_fs_residual ** 2)

    # AV at f/s interface
    av_pred_fs = av_net(inputs_fs)
    av_pred_fs_x = _grad(av_pred_fs, x_fs)
    av_fs_residual = ((-D_av * cc / lc) * av_pred_fs_x -
                      (k4_0 * torch.exp(alpha_av * u_pred_fs) -
                       U_av * phic / lc * u_pred_fs_x) * av_pred_fs * cc)
    av_fs_loss = torch.mean(av_fs_residual ** 2)

    # Potential at f/s interface
    u_fs_residual = ((eps_film * phic / lc * u_pred_fs_x) -
                     (eps_Ddl * phic * u_pred_fs))
    u_fs_loss = torch.mean(u_fs_residual ** 2)

    # Total boundary loss
    total_boundary_loss = cv_mf_loss + u_mf_loss + cv_fs_loss + av_fs_loss + u_fs_loss + av_mf_loss 

    individual_losses = {
        'cv_mf_bc': cv_mf_loss,
        'av_mf_bc': av_mf_loss,
        'u_mf_bc': u_mf_loss,
        'cv_fs_bc': cv_fs_loss,
        'av_fs_bc': av_fs_loss,
        'u_fs_bc': u_fs_loss,
    }

    if return_residuals:
        # Combine all residuals into single tensor for NTK computation

        residuals_dict = {'cv_mf_bc':cv_mf_residual, 'av_mf_bc':av_mf_residual, 'u_mf_bc':u_mf_residual, 
                    'cv_fs_bc':cv_fs_residual, 'av_fs_bc':av_mf_residual, 'u_fs_bc':u_fs_residual}
        combined_residuals = torch.cat([
            cv_mf_residual, cv_fs_residual,
            av_mf_residual, av_fs_residual,
        ])
        return total_boundary_loss, individual_losses, combined_residuals,residuals_dict
    else:
        return total_boundary_loss, individual_losses

def compute_initial_loss(x: torch.Tensor, t: torch.Tensor, E: torch.Tensor,
                         return_residuals: bool = False) -> Union[Tuple[torch.Tensor, Dict[str, torch.Tensor]],
Tuple[torch.Tensor, Dict[str, torch.Tensor], torch.Tensor]]:
    """
    Compute initial condition losses.

    **Initial Conditions (t̂ = 0):**

    **Film Thickness:**

    .. math::
        \\hat{L}(0) = \\frac{L_0}{\\hat{L}_c}

    **Cation Vacancy Concentration:**

    .. math::
        \\hat{c}_{cv}(\\hat{x}, 0) = 0

    .. math::
        \\frac{\\partial \\hat{c}_{cv}}{\\partial \\hat{t}}\\bigg|_{\\hat{t}=0} = 0

    **Anion Vacancy Concentration:**

    .. math::
        \\hat{c}_{av}(\\hat{x}, 0) = 0

    .. math::
        \\frac{\\partial \\hat{c}_{av}}{\\partial \\hat{t}}\\bigg|_{\\hat{t}=0} = 0

    **Potential Distribution:**

    .. math::
        \\hat{\\phi}(\\hat{x}, 0) = \\frac{\\hat{E}}{\\hat{\\phi}_c} - \\frac{10^7 \\hat{L}_c}{\\hat{\\phi}_c}\\hat{x}

    .. math::
        \\frac{\\partial \\hat{\\phi}}{\\partial \\hat{t}}\\bigg|_{\\hat{t}=0} = 0

    **Hole Concentration:**

    .. math::
        \\hat{c}_h(\\hat{x}, 0) = \\frac{c_{h0}}{\\hat{c}_{h,c}} = 1

    .. math::
        \\frac{\\partial \\hat{c}_h}{\\partial \\hat{t}}\\bigg|_{\\hat{t}=0} = 0

    Args:
        x: Spatial coordinates
        t: Time coordinates (should be zeros)
        E: Applied potential
        networks: NetworkManager instance
        physics: ElectrochemicalPhysics instance
        return_residuals: If True, also return raw residuals for NTK computation

    Returns:
        If return_residuals=False: Tuple of (total_initial_loss, individual_losses_dict)
        If return_residuals=True: Tuple of (total_initial_loss, individual_losses_dict, combined_residuals)
    """
    L_input = torch.cat([t, E], dim=1)
    L_initial_pred = L_net(L_input)
    inputs = torch.cat([x, t, E], dim=1)

    # Film thickness initial condition
    L_initial_residual = L_initial_pred - L_initial / lc
    L_initial_loss = torch.mean(L_initial_residual ** 2)

    # Cation vacancy initial conditions
    cv_initial_pred = cv_net(inputs)
    cv_initial_t = _grad(cv_initial_pred, t)
    cv_initial_residual = cv_initial_pred + cv_initial_t
    cv_initial_loss = torch.mean(cv_initial_pred**2) + torch.mean(cv_initial_t**2)

    # Anion vacancy initial conditions
    av_initial_pred = av_net(inputs)
    av_initial_t = _grad(av_initial_pred, t)
    av_initial_residual = av_initial_pred + av_initial_t
    av_initial_loss = torch.mean(av_initial_pred**2) + torch.mean(av_initial_t**2)

    # Potential initial conditions
    u_initial_pred = u_net(inputs)
    u_initial_t = _grad(u_initial_pred, t)
    poisson_initial_residual = (u_initial_pred - (
            (E / phic) - (1e7 * (lc / phic) * x))) + u_initial_t
    poisson_initial_loss = torch.mean((u_initial_pred - (
            (E / phic) - (1e7 * (lc / phic) * x)))**2) + torch.mean(u_initial_t**2)

    # Total initial loss
    total_initial_loss = cv_initial_loss + av_initial_loss + poisson_initial_loss + L_initial_loss

    individual_losses = {
        'cv_ic': cv_initial_loss,
        'av_ic': av_initial_loss,
        'poisson_ic': poisson_initial_loss,
        'L_ic': L_initial_loss
    }

    if return_residuals:
        # Combine all residuals into single tensor for NTK computation
        residual_dict = {'cv_ic':cv_initial_residual, 'av_ic':av_initial_residual,'poisson_ic':poisson_initial_residual, 'L_ic':L_initial_residual}
        combined_residuals = torch.cat([
            L_initial_residual,
            cv_initial_residual,
            av_initial_residual,
            poisson_initial_residual,
        ])
        return total_initial_loss, individual_losses, combined_residuals,residual_dict
    else:
        return total_initial_loss, individual_losses

def compute_film_physics_loss(t: torch.Tensor, E: torch.Tensor,
                              return_residuals: bool = False) -> Union[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
    """
    Compute film growth physics loss.

     **Film Growth Equation:**

    .. math::
        \\frac{dL}{dt} = \\Omega (k_2 - k_5)

    **Dimensionless Form:**

    .. math::
        \\frac{d\\hat{L}}{d\\hat{t}} = \\frac{\\hat{t}_c \\Omega}{\\hat{L}_c} (k_2 - k_5)


    Args:
        t: Time coordinates
        E: Applied potential
        networks: NetworkManager instance
        physics: ElectrochemicalPhysics instance
        return_residuals: If True, also return raw residuals for NTK computation

    Returns:
        If return_residuals=False: Film physics loss tensor
        If return_residuals=True: Tuple of (film_physics_loss, residuals)
    """
    inputs = torch.cat([t, E], dim=1)
    L_pred = L_net(inputs)

    # Get rate constants
    k1, k2, k3, k4, k5 = compute_rate_constants(t, E)

    # Compute predicted and physics-based dL/dt
    dl_dt_pred = _grad(L_pred, t)
    dL_dt_physics = (1 /lc) * tc * Omega * (k2 - k5) #k_2 being problematic, we see this in polarization curve would make sense if it happens here too

    # Compute residual
    film_residual = dl_dt_pred - dL_dt_physics
    film_loss = torch.mean(film_residual ** 2)

    if return_residuals:
        return film_loss, film_residual
    else:
        return film_loss


In [43]:
#Get the total loss with augmented lagrangian

#Create all the lambdas we need
l_cv_mf = torch.zeros(1024, device=device, requires_grad=True)
l_av_mf = torch.zeros(1024, device=device, requires_grad=True)
l_cv_fs = torch.zeros(1024, device=device, requires_grad=True)
l_av_fs = torch.zeros(1024, device=device, requires_grad=True)
l_u_fs = torch.zeros(1024, device=device, requires_grad=True)
l_cv_in = torch.zeros(1024, device=device, requires_grad=True)
l_av_in = torch.zeros(1024, device=device, requires_grad=True)
l_u_in = torch.zeros(1024, device=device, requires_grad=True)
l_film_in = torch.zeros(1024, device=device, requires_grad=True)
l_film = torch.zeros(2048, device=device, requires_grad=True)

def total_loss():
    beta = 500.0 # AL beta parameter
    #Investigate using one E for all the different samples, this might help a lot. Could do an easy fix here
    x_interior, t_interior, E_interior = sample_interior_points()
    x_boundary, t_boundary, E_boundary = sample_boundary_points()
    x_inital, t_initial, E_initial = sample_initial_points()
    t_film, E_film = sample_film_physics_points()

    total_interior_loss, individual_losses, residuals_interior = compute_interior_loss(x_interior,t_interior,E_interior,return_residuals=True)
    total_boundary_loss, boundary_losses, combined_residuals_bc,residuals_dict_bc = compute_boundary_loss(x_boundary,t_boundary,E_boundary,return_residuals=True)
    total_initial_loss, initial_losses, combined_residuals_ic,residuals_dict_ic = compute_initial_loss(x_inital,t_initial,E_initial,return_residuals=True)
    film_loss, film_residual = compute_film_physics_loss(t_film,E_film,return_residuals=True)

    # Augmented Lagrangian Loss: L = objective + β‖C‖² + ⟨λ,C⟩
    loss = total_interior_loss 

    # Boundary constraints
    loss += beta*boundary_losses["cv_mf_bc"] + (l_cv_mf*residuals_dict_bc["cv_mf_bc"]).mean()
    loss += beta*boundary_losses["av_mf_bc"] + (l_av_mf*residuals_dict_bc["av_mf_bc"]).mean()
    loss += beta*boundary_losses["cv_fs_bc"] + (l_cv_fs*residuals_dict_bc["cv_fs_bc"]).mean()
    loss += beta*boundary_losses["av_fs_bc"] + (l_av_fs*residuals_dict_bc["av_fs_bc"]).mean()
    loss += beta*boundary_losses["u_fs_bc"] + (l_u_fs*residuals_dict_bc["u_fs_bc"]).mean()

    # Initial constraints  
    loss += beta*initial_losses["cv_ic"] + (l_cv_in*residuals_dict_ic["cv_ic"]).mean()
    loss += beta*initial_losses["av_ic"] + (l_av_in*residuals_dict_ic["av_ic"]).mean()
    loss += beta*initial_losses["poisson_ic"] + (l_u_in*residuals_dict_ic["poisson_ic"]).mean()
    loss += beta*initial_losses["L_ic"] + (l_film_in*residuals_dict_ic["L_ic"]).mean()

    # Film physics constraint
    loss += beta*film_loss + (l_film*film_residual).mean()
    #should check if we need .view(-1)
    #could experiment with making film physics an objective vs a constraint. Worth discussing with Conrard

    return loss, total_interior_loss, total_boundary_loss, total_initial_loss, film_loss


In [44]:
#main training loop

optimizer=torch.optim.Adam([{'params': total_model_parameters}, {'params': l_cv_mf, 'lr':lbd_lr}, \
                                {'params': l_av_mf, 'lr':lbd_lr}, {'params': l_cv_fs, 'lr':lbd_lr},{'params': l_av_fs, 'lr':lbd_lr},{'params': l_u_fs, 'lr':lbd_lr}, 
                                {'params': l_cv_in, 'lr':lbd_lr},{'params': l_av_in, 'lr':lbd_lr},{'params': l_u_in, 'lr':lbd_lr},{'params': l_film_in, 'lr':lbd_lr},
                                {'params': l_film, 'lr':lbd_lr}], lr=lr)

loss_list = []
interior_loss_list = []
boundary_loss_list = []
initial_loss_list = []
film_loss_list = []

for step in tqdm(range(max_steps),desc="Training Status"):
    cv_net.train()
    av_net.train()
    u_net.train()
    L_net.train()
    optimizer.zero_grad()

    loss, interior_loss, boundary_loss, initial_loss, film_loss = total_loss()
    loss_total = interior_loss + boundary_loss + initial_loss + film_loss
    loss.backward()


    l_cv_mf.grad *= -1
    l_av_mf.grad *= -1
    l_cv_fs.grad *= -1
    l_av_fs.grad *= -1
    l_u_fs.grad *= -1
    l_cv_in.grad *= -1
    l_av_in.grad *= -1
    l_u_in.grad *= -1
    l_film_in.grad *= -1
    l_film.grad *= -1


    optimizer.step()
        
    loss_list.append(loss_total.item())
    interior_loss_list.append(interior_loss.item())
    boundary_loss_list.append(boundary_loss.item())
    initial_loss_list.append(initial_loss.item())
    film_loss_list.append(film_loss.item())

    if step % print_freq == 0:
        tqdm.write(f"Total:{loss_total}, interior:{interior_loss}, boundary:{boundary_loss}, initial:{initial_loss}, film:{film_loss} at step:{step}")


Training Status:   0%|          | 0/20000 [00:00<?, ?it/s]

Total:1996513.625, interior:1992528.5, boundary:0.004996717907488346, initial:1000.816650390625, film:2984.240478515625 at step:0


KeyboardInterrupt: 

In [ ]:
#Plotting

plt.figure(figsize=(12, 6))
window=100
# Raw loss
plt.plot(loss_list, alpha=0.3, color='lightblue', label='Raw Loss')

# Moving average for smoother trend
if len(loss_list) > window:
    moving_avg = np.convolve(loss_list, np.ones(window)/window, mode='valid')
    plt.plot(range(window-1, len(loss_list)), moving_avg, 
        color='red', linewidth=2, label=f'Moving Avg ({window})')

plt.yscale('log')
plt.xlabel('Training Step', fontsize=12)
plt.ylabel('Total Loss (log scale)', fontsize=12)
plt.title('PINNACLE Training Loss Evolution', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()
